<a href="https://colab.research.google.com/github/rooncal/Image-Classification/blob/main/ResNet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing libraries

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

from functools import reduce
import numpy as np

import sys
from google.colab import drive

# Downloading dataset

In [22]:
drive.mount('/content/drive', force_remount=True)

%cd drive/My\ Drive/assignment2/cs231n/datasets/
!bash get_datasets.sh
%cd /content

NUM_TRAIN = 49000

transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])


cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)

loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Mounted at /content/drive
/content/drive/My Drive/assignment2/cs231n/datasets
/content
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Defining device type as GPU

In [23]:
device = torch.device('cuda')

#The Model

###Defining a class for a residual block

In [37]:
class ResidualBlock(nn.Module):
  def __init__(self, in_channels, middle_channels, out_channels, stride=1):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=middle_channels,kernel_size=1,stride=stride)
    self.conv2 = nn.Conv2d(in_channels=middle_channels,out_channels=middle_channels,kernel_size=3,stride=1)
    self.conv3 = nn.Conv2d(in_channels=middle_channels,out_channels=out_channels,kernel_size=1,stride=1)


  def forward(self, x):
    temp = x
    x = self.conv1(x)
    x = nn.BatchNorm2d(num_features=x.view(1,-1).size(1))()
    x = nn.ReLU()(x)
    x = self.conv2(x)
    x = nn.BatchNorm2d(num_features=x.view(1,-1).size(1))()
    x = nn.ReLU()(x)
    x = self.conv3(x)
    x = nn.BatchNorm2d(num_features=x.view(1,-1).size(1))()
    x = nn.ReLU()(x)
    x += temp

In [38]:
class ResNet101(nn.Module):
  def forward(self, x):
    x = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3)(x)
    x = nn.MaxPool2d(kernel_size=3,stride=2)(x)

    x = ResidualBlock(in_channels=64, middle_channels=64, out_channels=256)(x)
    x = ResidualBlock(in_channels=256, middle_channels=64, out_channels=256)(x)
    x = ResidualBlock(in_channels=256, middle_channels=64, out_channels=256)(x)

    x = ResidualBlock(in_channels=256, middle_channels=128, out_channels=512,stride=2)(x)
    x = ResidualBlock(in_channels=512, middle_channels=128, out_channels=512)(x)
    x = ResidualBlock(in_channels=512, middle_channels=128, out_channels=512)(x)
    x = ResidualBlock(in_channels=512, middle_channels=128, out_channels=512)(x)

    x = ResidualBlock(in_channels=512, middle_channels=256, out_channels=1024, stride=2)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)
    x = ResidualBlock(in_channels=1024, middle_channels=256, out_channels=1024)(x)

    x = ResidualBlock(in_channels=1024, middle_channels=512, out_channels=2048, stride=2)(x)
    x = ResidualBlock(in_channels=2048, middle_channels=512, out_channels=2048)(x)
    x = ResidualBlock(in_channels=2048, middle_channels=512, out_channels=2048)(x)
    
    x = nn.AvgPool2d(kernel_size=3)(x)
    x = nn.Linear(in_features=(2048*7*7),out_features=10)(x)
    x = nn.Softmax()(x)
    return x

In [39]:
  x = torch.zeros((64, 3, 32, 32), dtype=torch.float32)  # minibatch size 64, image size [3, 32, 32]
  model = ResNet101()
  scores = model(x)
  print(scores.size())  # you should see [64, 10]

TypeError: ignored